# Proyecto End to End de Machine Learning 
### Viviendas en venta en Madrid


## 0. Librerías
 

In [18]:
# importación agrupada de librerías necesarias en este notebook
import pandas as pd
import numpy as np
import json

import sys
import os

from scipy import stats
from PIL import Image
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

#warnings.filterwarnings('ignore')

# Añado el directorio padre (del que está este notebook) a sys.path
sys.path.append(os.path.abspath('../'))
from scripts.utils_agv import ini_inspec, crear_tabla_resumen, categoricas, numericas

## 4. Compresión de variables

In [38]:
# carga de los datos guardados en el anterior paso
df = pd.read_csv('../data/processed/ide_viv_limpieza0_2025-03-11.csv', index_col='propertyCode')
df.head(3)

,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,address,district,...,status,hasLift,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,topPlus,parkingSpace
propertyCode,,,,,,,,,,,,,,,,,,,,,
107526421,43,3,1095000.0,flat,146.0,True,3,2,calle de las Fuentes,Centro,...,good,True,7500.0,{'typology': 'flat'},"{'subtitle': 'Sol, Madrid', 'title': 'Piso en ...",True,True,False,True,NaN
107246640,27,2,1195000.0,flat,134.0,True,3,3,calle del Divino Pastor,Centro,...,good,True,8918.0,{'typology': 'flat'},"{'subtitle': 'Malasaña-Universidad, Madrid', '...",True,True,True,True,NaN
106773918,33,3,790000.0,flat,148.0,True,2,2,Barrio Lavapiés-Embajadores,Centro,...,good,True,5338.0,{'typology': 'flat'},"{'subtitle': 'Lavapiés-Embajadores, Madrid', '...",True,True,False,True,NaN


In [32]:
df['detailedType'].unique()

array(["{'typology': 'flat'}",
       "{'typology': 'flat', 'subTypology': 'studio'}",
       "{'typology': 'flat', 'subTypology': 'penthouse'}",
       "{'typology': 'flat', 'subTypology': 'duplex'}"], dtype=object)

In [33]:
df['suggestedTexts'].unique()

array(["{'subtitle': 'Sol, Madrid', 'title': 'Piso en calle de las Fuentes'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle del Divino Pastor'}",
       "{'subtitle': 'Lavapiés-Embajadores, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Sol, Madrid', 'title': 'Piso en Carrera de San Jerónimo'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Estudio en calle de San Andrés'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Ático en calle de Fuencarral'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle de Fuencarral'}",
       "{'subtitle': 'Palacio, Madrid', 'title': 'Dúplex en calle de Santiago'}",
       "{'subtitle': 'Chueca-Justicia, Madrid', 'title': 'Piso en calle de Apodaca'}",
       "{'subtitle': 'Sol, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 'title': 'Piso en calle del Espíritu Santo'}",
       "{'subtitle': 'Malasaña-Universidad, Madrid', 't

In [16]:
df['parkingSpace'].unique()

array([nan,
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 80000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 18000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 50000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 60000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 100000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 45000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 40000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 150.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice':

Voy a separar en columnas aquellas cuyos datos son dicionarios:


In [43]:
# Conversión columnas que contienen diccionarios en múltiples columnas
# Función para convertir strings en diccionarios JSON

# 🔍 Función para convertir JSON en diccionario (manejando errores)
def parse_json(x):
    if isinstance(x, str) and x.strip():  # Aseguramos que es string no vacío
        try:
            x = x.replace("'", "\"")  # Corrige comillas simples a dobles
            return json.loads(x)  # Intenta convertir a JSON
        except json.JSONDecodeError:
            return {}  # Si falla, devuelve diccionario vacío
    return {}  # Si no es string, devuelve diccionario vacío

# Aplicamos la conversión a las columnas de interés
df['suggestedTexts'] = df['suggestedTexts'].apply(parse_json)
df['detailedType'] = df['detailedType'].apply(parse_json)
df['parkingSpace'] = df['parkingSpace'].apply(lambda x: parse_json(x) if pd.notna(x) else {})

# 🔍 Convertimos los diccionarios en columnas nuevas
df_suggested = pd.json_normalize(df['suggestedTexts']).add_prefix('suggestedTexts_')
df_detailed = pd.json_normalize(df['detailedType']).add_prefix('detailedType_')
df_parking = pd.json_normalize(df['parkingSpace']).add_prefix('parkingSpace_')

# 🔍 Comprobamos si las columnas de parking están vacías
print("df_parking:")
print(df_parking)


df_parking:
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[1792 rows x 0 columns]


In [29]:
df_parking

""
0
1
2
3
4
...
1787
1788
1789
1790


In [11]:
ini_inspec(df)

=== TAMAÑO Y ESTRUCTURA DE LOS DATOS ===
Número total de registros (filas): 1792
Número de columnas: 25
Uso de memoria: 302.75 KB


=== TIPOS DE DATOS Y NOMBRES DE COLUMNAS ===
numPhotos           int64
floor              object
price             float64
propertyType       object
size              float64
exterior           object
rooms               int64
bathrooms           int64
address            object
district           object
neighborhood       object
latitude          float64
longitude         float64
description        object
hasVideo             bool
status             object
hasLift            object
priceByArea       float64
detailedType       object
suggestedTexts     object
hasPlan              bool
has3DTour            bool
has360               bool
topPlus              bool
parkingSpace       object
dtype: object


Información detallada del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1792 entries, 107526421 to 107518858
Data columns (total 25 columns):
 #   

A continuación, una rápida analítica de cada una de las variables.
1. **Variable**: nombre variable/alias
2. **Data type**: cualitativa, cuantitativa, ordinal, continua...¿?
3. **Segmento**: clasificar las variables según su significado. Si son variables demográficas, económicas, identificadores, tiempo...
4. **Expectativas**: un pequeño indicador personal de si resultará útil la variable. ¿Necesito esta variable para la solución? ¿Cómo de importante será esta variable? ¿Esta info la recoge otra variable ya vista?
5. **Conclusiones**: después del análisis anterior, llegar a unas conclusiones sobre la importancia de la variable.

|Variable |Dtype |tipo |faltantes |segmento |expectativas |conclusiones|
|--|--|--|--|--|--|--|
|||||unidades |descripción ||
|propertyCode(id)| int64| entero
|numPhotos|int64 | entero|
|floor |object | entero|
|price (target)| float64| continuo
|propertyType| object |categórico
|size| float64| continuo|
|exterior| object| booleano |
|rooms| int64| entero|
|bathrooms| int64| entero|
|address| object| categórico|
|district| object| categórico|
|neighborhood| object| categórico|
|latitude|float64 | continuo|
|longitude|float64 | continuo|
|description| object| categórico|
|hasVideo| bool| booleano |
|status| object| categórico|
|hasLift|  object| booleano |
|priceByArea| float64 | continuo|
|detailedType| object| categórico|
|suggestedTexts|object| categórico|
|hasPlan| bool| booleano |
|has3DTour|bool| booleano |
|has360| bool| booleano |
|topPlus| bool| booleano |
|parkingSpace|object| categórico|

## 15. División en train y test
(resevar una porción de los datos obtenidos para probar nuestros modelos)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'],
                                                    test_size=0.2,
                                                    random_state=42)